In [1]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no

In [2]:
# !pip install pyproj

In [3]:
# 폰트 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 1. 라이브러리 설치 및 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Transformer
from sklearn.preprocessing import StandardScaler

In [6]:
# dtype_dict = {
#    '폐업일자': str,
#    '휴업시작일자': str,
#    '도로명전체주소': str,
#    '업태구분명': str,
#    '사업장명': str,
# }

feature_cols  = [
    "인허가일자",
    "인허가취소일자",
    "영업상태구분코드",
    "영업상태명",
    "상세영업상태코드",
    "상세영업상태명",
    "폐업일자",
    "휴업시작일자",
    "휴업종료일자",
    "재개업일자",
    "소재지전체주소",
    "도로명전체주소",
    "업태구분명",
    '좌표정보X(EPSG5174)',
    '좌표정보Y(EPSG5174)',
    "사업장명",
    "데이터갱신일자",
    "소재지면적",
    # "시설총규모"
]
date_cols = ['인허가일자', '인허가취소일자', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '데이터갱신일자']
target_si_names = ['화성시','안산시','수원시','안양시','용인시', '부천시']
target_si = '|'.join(target_si_names)
cafe_types = ['커피숍', '카페', '까페', '커피전문점', '다방']
franchise_list = [
    "감성커피", "더벤티", "더카페", "드롭탑", "만랩커피", "빽다방", "셀렉토커피", "스타벅스", "이디야커피", "dal.komm",
    "달.콤커피", "메가엠지씨커피", "우지커피", "백억커피", "카페일리터", "매머드익스프레스", "테라커피", "텐퍼센트커피",
    "TENPERCENT COFFEE", "고망고", "카페베네", "커피마마", "커피빈", "커피에반하다", "컴포즈커피", "탐앤탐스", "토프레소",
    "파스쿠찌", "하삼동커피", "할리스", "투썸플레이스", "공차", "천씨씨", "그라찌에", "빈스빈스", "카페보니또", "커피베이",
    "엔제리너스", "전광수커피", "커피나무", "커피스미스", "요거프레소", "쥬씨", "네스프레소", "쮸커커피", "코나카페",
    "폴바셋", "카페루트", "콩볶는커피쟁이", "골드빈Ⅱ", "카페더베어", "카페모어", "반달커피", "카페모니", "카페소소",
    "카페아이엔지", "카페168", "일리카페", "카페드랍탑", "마북173", "블루뮤트", "더폴", "카페빔", "더리터", "벤티프레소",
    "타르트봉봉", "카페시그니쳐", "달콤커피", "달리는커피", "매머드커피", "투썸커피", "탐앤탐스커피", "더리터", "메가커피",
    "커피명가", "커피스미스", "더착한커피", "벤티프레소", "텍퍼센트커피", "아마스빈", "투썸커피"
]

props = {
    # 'dtype': dtype_dict,
    'usecols': feature_cols
}

df_list = []
for i in range(len(target_si_names) * 2):
  일반_content = pd.read_excel(f"/content/drive/MyDrive/2nd_미니_프로젝트/07_24_04_P_일반음식점 ({i}).xlsx", **props)
  휴게_content = pd.read_excel(f"/content/drive/MyDrive/2nd_미니_프로젝트/07_24_05_P_휴게음식점 ({i}).xlsx", **props)

  df_list.append(일반_content)
  df_list.append(휴게_content)

df = pd.concat(df_list, ignore_index=True)

df = df[feature_cols]
df = df[
    df['업태구분명'].isin(cafe_types) &
    df['소재지전체주소'].str.contains(target_si, na=False)
].copy()

# epsg5174 기준 좌표를 lon, lat으로 수정
transformer = Transformer.from_crs("epsg:5174", "epsg:4326", always_xy=True)
df['lon'], df['lat'] = transformer.transform(df['좌표정보X(EPSG5174)'].values, df['좌표정보Y(EPSG5174)'].values)
df = df.drop(columns=['좌표정보X(EPSG5174)', '좌표정보Y(EPSG5174)'])

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

cutoff_date = pd.to_datetime('2016-01-01')
df = df[(df['폐업일자'] >= cutoff_date) | (df['폐업일자'].isna())]

df['도로명전체주소'] = df['도로명전체주소'].fillna(df['소재지전체주소'])
df['소재지면적'] = df['소재지면적'].fillna(df['소재지면적'].mean())

df['폐업여부'] = df['영업상태명'].apply(lambda x: 1 if x == '폐업' else 0)
df.loc[(df['영업상태명'] == '폐업') & (df['폐업일자'].isna()), '폐업일자'] = df['데이터갱신일자']
df['폐업연도'] = np.where(
    (df['영업상태명'] == '폐업') & (df['폐업일자'].notna()),
    df['폐업일자'].dt.year,
    None
)
df['인허가연도'] = df['인허가일자'].dt.year
df['영업기간(일)'] = (df['폐업일자'] - df['인허가일자']).dt.days

df['직원수'] = (df['소재지면적'] / 30).round(2)

df['시군명'] = df['소재지전체주소'].str.extract(r'([가-힣]+시)')

df['프랜차이즈'] = df['사업장명'].apply(
    lambda x: 1 if any(franchise in x for franchise in franchise_list) else 0
)

print('영업', df[df['상세영업상태명'] == '영업'].shape[0])
print('영업X', df[df['상세영업상태명'] != '영업'].shape[0])

# print(df.isna().sum())
# print()
print(df.shape)

df.to_csv('일반_휴게_통합_카페_데이터.csv', index=False, encoding='utf-8-sig')

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

영업 7159
영업X 8800
(15959, 25)


In [7]:
temp_df = pd.read_csv("/content/일반_휴게_통합_카페_데이터.csv", encoding="utf-8")
print(temp_df.isna().sum())

인허가일자           0
인허가취소일자     15959
영업상태구분코드        0
영업상태명           0
상세영업상태코드        0
상세영업상태명         0
폐업일자         7159
휴업시작일자      15959
휴업종료일자      15959
재개업일자       15959
소재지전체주소         0
도로명전체주소         0
업태구분명           0
사업장명            0
데이터갱신일자         0
소재지면적           0
lon           880
lat           880
폐업여부            0
폐업연도         7159
인허가연도           0
영업기간(일)      7159
직원수             0
시군명             0
프랜차이즈           0
dtype: int64
